# 第10课：降维技术

## 学习目标
- 理解降维的意义和应用场景
- 掌握 PCA 主成分分析
- 掌握 t-SNE 可视化
- 了解 UMAP 降维方法
- 学习特征选择方法

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits, load_iris, fetch_olivetti_faces
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

## 1. 为什么需要降维？

**问题**：
- 高维数据难以可视化
- 维度灾难：高维空间数据稀疏
- 特征冗余：很多特征相关性高
- 计算成本高

**降维的好处**：
- 数据可视化
- 降低计算成本
- 去除噪声
- 缓解过拟合

In [ ]:
# 加载手写数字数据集
digits = load_digits()
X, y = digits.data, digits.target

print(f"数据形状: {X.shape}")
print(f"每个样本是 8x8 = 64 维的图像")
print(f"类别数: {len(np.unique(y))}")

In [ ]:
# 可视化部分样本
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

for i, ax in enumerate(axes.flat):
    ax.imshow(X[i].reshape(8, 8), cmap='gray')
    ax.set_title(f'Label: {y[i]}')
    ax.axis('off')

plt.suptitle('Digit Samples')
plt.tight_layout()
plt.show()

## 2. PCA 主成分分析

PCA (Principal Component Analysis) 是最常用的线性降维方法。

**核心思想**：
- 找到方差最大的方向（主成分）
- 将数据投影到这些主成分上
- 保留最重要的成分，丢弃次要的

In [ ]:
# 标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA 降维到 2D
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

print(f"原始维度: {X.shape[1]}")
print(f"降维后维度: {X_pca_2d.shape[1]}")
print(f"解释方差比例: {pca_2d.explained_variance_ratio_}")
print(f"累计解释方差: {sum(pca_2d.explained_variance_ratio_):.4f}")

In [ ]:
# 可视化 PCA 结果
plt.figure(figsize=(10, 8))

scatter = plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y, cmap='tab10', alpha=0.7)
plt.colorbar(scatter, label='Digit')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('PCA Visualization of Digits Dataset (2D)')
plt.show()

In [ ]:
# 分析需要多少个主成分
pca_full = PCA()
pca_full.fit(X_scaled)

# 累计解释方差
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

plt.figure(figsize=(12, 5))

# 单个成分解释方差
plt.subplot(1, 2, 1)
plt.bar(range(1, 21), pca_full.explained_variance_ratio_[:20])
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Variance Explained by Each Component')

# 累计解释方差
plt.subplot(1, 2, 2)
plt.plot(range(1, len(cumulative_variance)+1), cumulative_variance, 'b-o')
plt.axhline(y=0.9, color='r', linestyle='--', label='90% variance')
plt.axhline(y=0.95, color='g', linestyle='--', label='95% variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 找到保留 90% 和 95% 方差所需的成分数
n_90 = np.argmax(cumulative_variance >= 0.9) + 1
n_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"保留 90% 方差需要 {n_90} 个成分")
print(f"保留 95% 方差需要 {n_95} 个成分")

## 3. PCA 用于分类任务

In [ ]:
# 比较不同维度对分类效果的影响
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

n_components_list = [2, 5, 10, 20, 30, 40, 50, 64]
accuracies = []

for n in n_components_list:
    if n < X.shape[1]:
        pca = PCA(n_components=n)
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)
    else:
        X_train_pca = X_train
        X_test_pca = X_test
    
    clf = LogisticRegression(max_iter=1000, random_state=42)
    clf.fit(X_train_pca, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test_pca))
    accuracies.append(acc)
    print(f"n_components={n}: 准确率={acc:.4f}")

plt.figure(figsize=(10, 5))
plt.plot(n_components_list, accuracies, 'b-o')
plt.xlabel('Number of Components')
plt.ylabel('Accuracy')
plt.title('Classification Accuracy vs Number of PCA Components')
plt.grid(True, alpha=0.3)
plt.show()

## 4. t-SNE 可视化

t-SNE (t-Distributed Stochastic Neighbor Embedding) 是一种非线性降维方法，特别适合数据可视化。

**特点**：
- 保持局部结构
- 适合可视化聚类
- 计算成本较高
- 不适合用于新数据转换

In [ ]:
# t-SNE 降维
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)

print(f"t-SNE 输出形状: {X_tsne.shape}")

In [ ]:
# 对比 PCA 和 t-SNE
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# PCA
scatter1 = axes[0].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y, cmap='tab10', alpha=0.7, s=10)
axes[0].set_xlabel('Component 1')
axes[0].set_ylabel('Component 2')
axes[0].set_title('PCA')

# t-SNE
scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='tab10', alpha=0.7, s=10)
axes[1].set_xlabel('Component 1')
axes[1].set_ylabel('Component 2')
axes[1].set_title('t-SNE')

plt.colorbar(scatter2, ax=axes[1], label='Digit')
plt.tight_layout()
plt.show()

In [ ]:
# 不同 perplexity 的影响
perplexities = [5, 30, 50, 100]

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

for idx, perp in enumerate(perplexities):
    tsne = TSNE(n_components=2, random_state=42, perplexity=perp)
    X_tsne_temp = tsne.fit_transform(X_scaled)
    
    axes[idx].scatter(X_tsne_temp[:, 0], X_tsne_temp[:, 1], c=y, cmap='tab10', alpha=0.7, s=10)
    axes[idx].set_title(f'perplexity = {perp}')
    axes[idx].set_xlabel('Component 1')
    axes[idx].set_ylabel('Component 2')

plt.suptitle('t-SNE with Different Perplexity Values')
plt.tight_layout()
plt.show()

## 5. UMAP 降维

UMAP (Uniform Manifold Approximation and Projection) 是新一代的降维方法。

**特点**：
- 比 t-SNE 更快
- 可以用于新数据转换
- 更好地保持全局结构
- 支持监督/半监督降维

In [ ]:
# 安装: pip install umap-learn
try:
    import umap
    UMAP_AVAILABLE = True
    print(f"UMAP 已安装")
except ImportError:
    UMAP_AVAILABLE = False
    print("请先安装 umap: pip install umap-learn")

In [ ]:
if UMAP_AVAILABLE:
    import umap

    # UMAP 降维
    umap_reducer = umap.UMAP(n_components=2, random_state=42)
    X_umap = umap_reducer.fit_transform(X_scaled)

    print(f"UMAP 输出形状: {X_umap.shape}")
else:
    print("跳过 UMAP（未安装）")

In [ ]:
if UMAP_AVAILABLE:
    # 三种方法对比
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # PCA
    axes[0].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y, cmap='tab10', alpha=0.7, s=10)
    axes[0].set_title('PCA')

    # t-SNE
    axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='tab10', alpha=0.7, s=10)
    axes[1].set_title('t-SNE')

    # UMAP
    scatter = axes[2].scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='tab10', alpha=0.7, s=10)
    axes[2].set_title('UMAP')

    plt.colorbar(scatter, ax=axes[2], label='Digit')
    plt.suptitle('Comparison of Dimensionality Reduction Methods')
    plt.tight_layout()
    plt.show()
else:
    # 只对比 PCA 和 t-SNE
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    axes[0].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y, cmap='tab10', alpha=0.7, s=10)
    axes[0].set_title('PCA')

    scatter = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='tab10', alpha=0.7, s=10)
    axes[1].set_title('t-SNE')

    plt.colorbar(scatter, ax=axes[1], label='Digit')
    plt.suptitle('Comparison of Dimensionality Reduction Methods (UMAP not available)')
    plt.tight_layout()
    plt.show()

## 6. 核 PCA (Kernel PCA)

用于非线性降维的 PCA 变体

In [ ]:
# 加载鸢尾花数据
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

# 标准化
X_iris_scaled = StandardScaler().fit_transform(X_iris)

# 比较不同核的 Kernel PCA
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

fig, axes = plt.subplots(1, 4, figsize=(18, 4))

for idx, kernel in enumerate(kernels):
    kpca = KernelPCA(n_components=2, kernel=kernel, gamma=0.1)
    X_kpca = kpca.fit_transform(X_iris_scaled)
    
    axes[idx].scatter(X_kpca[:, 0], X_kpca[:, 1], c=y_iris, cmap='viridis', alpha=0.8)
    axes[idx].set_title(f'Kernel: {kernel}')
    axes[idx].set_xlabel('Component 1')
    axes[idx].set_ylabel('Component 2')

plt.suptitle('Kernel PCA with Different Kernels')
plt.tight_layout()
plt.show()

## 7. 特征选择 vs 降维

- **降维**：创建新特征（原特征的组合）
- **特征选择**：选择原有特征的子集

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# 使用手写数字数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 方法1: SelectKBest (基于统计测试)
selector_f = SelectKBest(f_classif, k=20)
X_train_f = selector_f.fit_transform(X_train, y_train)
X_test_f = selector_f.transform(X_test)

# 方法2: 基于互信息
selector_mi = SelectKBest(mutual_info_classif, k=20)
X_train_mi = selector_mi.fit_transform(X_train, y_train)
X_test_mi = selector_mi.transform(X_test)

# 方法3: RFE (递归特征消除)
rf = RandomForestClassifier(n_estimators=50, random_state=42)
rfe = RFE(rf, n_features_to_select=20)
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)

print(f"原始特征数: {X_train.shape[1]}")
print(f"选择后特征数: {X_train_f.shape[1]}")

In [ ]:
# 比较不同方法的分类效果
clf = LogisticRegression(max_iter=1000, random_state=42)

# 原始数据
clf.fit(X_train, y_train)
acc_original = accuracy_score(y_test, clf.predict(X_test))

# F-score 选择
clf.fit(X_train_f, y_train)
acc_f = accuracy_score(y_test, clf.predict(X_test_f))

# 互信息选择
clf.fit(X_train_mi, y_train)
acc_mi = accuracy_score(y_test, clf.predict(X_test_mi))

# RFE 选择
clf.fit(X_train_rfe, y_train)
acc_rfe = accuracy_score(y_test, clf.predict(X_test_rfe))

# PCA 降维
pca = PCA(n_components=20)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
clf.fit(X_train_pca, y_train)
acc_pca = accuracy_score(y_test, clf.predict(X_test_pca))

results = pd.DataFrame({
    'Method': ['Original (64)', 'F-score (20)', 'Mutual Info (20)', 'RFE (20)', 'PCA (20)'],
    'Accuracy': [acc_original, acc_f, acc_mi, acc_rfe, acc_pca]
})
print(results.to_string(index=False))

## 8. 人脸数据降维实战

> **注意**：以下代码使用 `fetch_olivetti_faces()` 加载 Olivetti 人脸数据集，首次运行时会自动从网络下载（约 3MB）。

In [ ]:
# 加载人脸数据
faces = fetch_olivetti_faces()
X_faces = faces.data
y_faces = faces.target

print(f"数据形状: {X_faces.shape}")
print(f"每张人脸是 64x64 = 4096 维")
print(f"人数: {len(np.unique(y_faces))}")

In [ ]:
# 显示部分人脸
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

for i, ax in enumerate(axes.flat):
    ax.imshow(X_faces[i*10].reshape(64, 64), cmap='gray')
    ax.set_title(f'Person {y_faces[i*10]}')
    ax.axis('off')

plt.suptitle('Sample Faces')
plt.tight_layout()
plt.show()

In [ ]:
# PCA 降维并可视化主成分（特征脸）
pca_faces = PCA(n_components=50)
X_faces_pca = pca_faces.fit_transform(X_faces)

# 显示前 10 个特征脸
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

for i, ax in enumerate(axes.flat):
    ax.imshow(pca_faces.components_[i].reshape(64, 64), cmap='gray')
    ax.set_title(f'PC {i+1}')
    ax.axis('off')

plt.suptitle('Top 10 Eigenfaces (Principal Components)')
plt.tight_layout()
plt.show()

In [ ]:
# 人脸重建
n_components_list = [5, 10, 25, 50, 100, 200]

fig, axes = plt.subplots(2, len(n_components_list)+1, figsize=(16, 5))

# 选择两张人脸
face_indices = [0, 100]

for row, face_idx in enumerate(face_indices):
    # 原图
    axes[row, 0].imshow(X_faces[face_idx].reshape(64, 64), cmap='gray')
    axes[row, 0].set_title('Original')
    axes[row, 0].axis('off')
    
    # 不同成分数的重建
    for col, n_comp in enumerate(n_components_list):
        pca_temp = PCA(n_components=n_comp)
        X_temp = pca_temp.fit_transform(X_faces)
        X_reconstructed = pca_temp.inverse_transform(X_temp)
        
        axes[row, col+1].imshow(X_reconstructed[face_idx].reshape(64, 64), cmap='gray')
        axes[row, col+1].set_title(f'n={n_comp}')
        axes[row, col+1].axis('off')

plt.suptitle('Face Reconstruction with Different Number of Components')
plt.tight_layout()
plt.show()

## 9. 练习题

### 练习1：Fashion-MNIST 降维
使用 t-SNE 对 Fashion-MNIST 数据集进行可视化

In [ ]:
# 提示: 可以使用 sklearn.datasets 或 keras.datasets 加载数据
# 在这里编写代码


### 练习2：PCA + 分类
找出保持 95% 分类准确率所需的最少主成分数

In [ ]:
# 在这里编写代码


## 10. 本课小结

### 降维方法对比

| 方法 | 类型 | 适用场景 | 特点 |
|------|------|----------|------|
| PCA | 线性 | 通用降维 | 快速，可解释 |
| Kernel PCA | 非线性 | 非线性数据 | 处理非线性关系 |
| t-SNE | 非线性 | 可视化 | 保持局部结构 |
| UMAP | 非线性 | 可视化/降维 | 快速，保持全局结构 |

### 使用建议

1. **数据可视化**：优先使用 UMAP 或 t-SNE
2. **特征压缩**：使用 PCA
3. **分类任务预处理**：PCA 保留 95% 方差
4. **非线性数据**：考虑 Kernel PCA 或 UMAP

### 重要参数

- **PCA**: n_components（成分数或方差比例）
- **t-SNE**: perplexity（邻域大小，5-50）
- **UMAP**: n_neighbors、min_dist